In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import itertools
import json
import sys
# sys.path.append("..")
from utils import data_proc_tools as dpt
from utils import plot_tools as pt
from utils.custom_metrics import recall, precision, binary_accuracy
from utils.custom_metrics import recall_np, precision_np, binary_accuracy_np, multilabel_confusion_matrix
from utils.rnn_textsum_models import HierSeq2Seq
import random
random.seed(42)
random_state=1000
pd.set_option('display.max_colwidth', -1)
import pylab

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [3]:
dir = '/vol/medic02/users/ag6516/radiology_report_summarisation/'
data_dir = dir + 'data/'

aug = 'aug'

model_output_dir = dir + 'trained_models/hierseq2seq/'

train_df = pd.read_pickle(data_dir + 'train/{}_train.pkl'.format(aug))
val_df = pd.read_pickle(data_dir + 'val/val.pkl')

## Load and prepare sequence data for training

In [4]:
train_df.head()

,examid,report,all_mesh,single_mesh
0,CXR1000_IM-0003,"[increased, opacity, within, right, upper, lobe, possible, mass, associated, area, atelectasis, focal, consolidation, ., cardiac, silhouette, within, normal, limits, ., opacity, left, midlung, overlying, posterior, left, 5th, rib, may, represent, focal, airspace, disease, ., increased, opacity, right, upper, lobe, associated, atelectasis, may, represent, focal, consolidation, mass, lesion, atelectasis, ., recommend, chest, ct, evaluation, ., opacity, overlying, left, 5th, rib, may, represent, focal, airspace, disease]","[opacity, lung, lingula, opacity, lung, upper_lobe, right, pulmonary_atelectasis, upper_lobe, right]","[opacity, lung, upper_lobe, right]"
1,CXR1001_IM-0004,"[interstitial, markings, diffusely, prominent, throughout, lungs, ., heart, size, normal, ., pulmonary, normal, ., diffuse, fibrosis]","[diffuse, markings, lung, bilateral, interstitial, diffuse, prominent]","[markings, lung, bilateral, interstitial, diffuse, prominent]"
2,CXR1002_IM-0004,"[status, post, left, mastectomy, ., heart, size, normal, ., lungs, clear]",[left],[left]
3,CXR1003_IM-0005,"[heart, size, pulmonary, vascularity, appear, within, normal, limits, ., retrocardiac, soft, tissue, density, present, ., appears, air, within, suggest, represents, hiatal, hernia, ., vascular, calcification, noted, ., calcified, granuloma, seen, ., interval, development, bandlike, opacity, left, lung, base, ., may, represent, atelectasis, ., osteopenia, present, spine, ., retrocardiac, soft, tissue, density, ., appearance, suggests, hiatal, hernia, ., left, base, bandlike, opacity, ., appearance, suggests, atelectasis]","[bone_diseases_metabolic, spine, calcified_granuloma, calcinosis, blood_vessels, density, retrocardiac, opacity, lung, base, left]","[opacity, lung, base, left]"
4,CXR1004_IM-0005,"[heart, ,, pulmonary, mediastinum, within, normal, limits, ., aorta, tortuous, ectatic, ., degenerative, changes, acromioclavicular, joints, ., degenerative, changes, spine, ., ivc, identified]","[aorta, tortuous, catheters_indwelling, shoulder, bilateral, degenerative, spine, degenerative]","[shoulder, bilateral, degenerative]"


In [6]:
# prepend and append start and end tokens to mesh captions and text reports
start_token = 'start'
end_token = 'end'
unknown_token = '**unknown**'
max_mesh_length = 13 # avg. + 1std. + start + end
max_num_words = 11 # avg. + 1std. + start + end, max number of words per sentence
max_num_sentences = 6 # avg. + 1std. + start + end, max number of sentences per report

In [7]:
def split_pad_report(report, max_num_sentences, max_num_words, start_token, end_token):
    pad_seq = [end_token for x in range(max_num_words)]
    r = ' '.join(report)
    sentences = r.split(' . ')
    padded_sentences = []
    for sen in sentences:
        words = sen.split(' ')
        padded_sentence = dpt.pad_sequence(words, max_num_words, start_token, end_token)
        padded_sentences.append(padded_sentence)
    if len(padded_sentences) >= max_num_sentences:
        padded_sentences = padded_sentences[:max_num_sentences]
    else:
        while len(padded_sentences) < max_num_sentences:
            padded_sentences.append(pad_seq)
    return padded_sentences

In [8]:
train_df['pad_mesh_caption'] = train_df.all_mesh.apply(lambda x: dpt.pad_sequence(x, max_mesh_length, start_token, end_token))
train_df['pad_text_report'] = train_df.report.apply(lambda x: split_pad_report(x, max_num_sentences, max_num_words, start_token, end_token))

val_df['pad_mesh_caption'] = val_df.all_mesh.apply(lambda x: dpt.pad_sequence(x, max_mesh_length, start_token, end_token))
val_df['pad_text_report'] = val_df.report.apply(lambda x: split_pad_report(x, max_num_sentences, max_num_words, start_token, end_token))

## Vectorise text reports and mesh captions

In [9]:
train_mesh = list(train_df.pad_mesh_caption)
train_reports = list(train_df.pad_text_report)

# vectorize train mesh captions
dpt.mesh_to_vectors(train_mesh, dicts_dir=data_dir+'dicts/', 
                    load_dicts=True, save=True, 
                    output_dir=data_dir+'train/')

# vectorise train reports
vec_reports = []
for report in train_reports:
    vec = dpt.reports_to_vectors(report, 
                                 dicts_dir=data_dir+'dicts/', 
                                 load_dicts=True, 
                                 output_dir=data_dir+'train/')
    vec_reports.append(vec)
vec_reports = np.array(vec_reports)
np.save(data_dir+'train/' + 'sent_token_ids_array.npy', vec_reports)

In [10]:
val_reports = list(val_df.pad_text_report)
val_mesh = list(val_df.pad_mesh_caption)

# vectorise val mesh using the same dict as created for train
dpt.mesh_to_vectors(val_mesh, dicts_dir=data_dir+'dicts/', 
                    load_dicts=True, save=True, 
                    output_dir=data_dir+'val/')

# vectorise val reports
vec_reports = []
for report in val_reports:
    vec = dpt.reports_to_vectors(report, 
                                 dicts_dir=data_dir+'dicts/', 
                                 load_dicts=True, 
                                 output_dir=data_dir+'train/')
    vec_reports.append(vec)
vec_reports = np.array(vec_reports)
np.save(data_dir+'val/' + 'sent_token_ids_array.npy', vec_reports)

In [11]:
word_to_id, id_to_word = dpt.load_report_dicts(data_dir+'dicts/')
mesh_to_id, id_to_mesh = dpt.load_mesh_dicts(data_dir+'dicts/')

report_vocab_length = len(word_to_id)
mesh_vocab_length = len(mesh_to_id)

In [12]:
report_vocab_length, mesh_vocab_length

(1475, 128)

In [13]:
# Create arrays of indixes for input sentences, output entities and shifted output entities (t-1)
train_token_ids_array = np.load(data_dir + 'train/sent_token_ids_array.npy')
train_mesh_ids_array = np.load(data_dir + 'train/mesh_ids_array.npy')
train_mesh_ids_array_shifted =[np.concatenate((mesh_to_id[start_token], t[:-1]), axis=None) for t in train_mesh_ids_array]
train_mesh_ids_array_shifted = np.asarray(train_mesh_ids_array_shifted)

val_token_ids_array = np.load(data_dir + 'val/sent_token_ids_array.npy')
val_mesh_ids_array = np.load(data_dir + 'val/mesh_ids_array.npy')
val_mesh_ids_array_shifted = [np.concatenate((mesh_to_id[start_token], t[:-1]), axis=None) for t in val_mesh_ids_array]
val_mesh_ids_array_shifted = np.asarray(val_mesh_ids_array_shifted)

In [14]:
# one-hot-encode
one_hot_reports_train = []
for report in train_token_ids_array:
    one_hot_reports_train.append(dpt.one_hot_sequence(report, report_vocab_length))
one_hot_reports_train = np.array(one_hot_reports_train)
#dpt.one_hot_sequence(train_token_ids_array, report_vocab_length)
one_hot_mesh_train = dpt.one_hot_sequence(train_mesh_ids_array, mesh_vocab_length)
one_hot_mesh_shifted_train = dpt.one_hot_sequence(train_mesh_ids_array_shifted, mesh_vocab_length)

one_hot_reports_val = []
for report in val_token_ids_array:
    one_hot_reports_val.append(dpt.one_hot_sequence(report, report_vocab_length))
one_hot_reports_train = np.array(one_hot_reports_train)
one_hot_mesh_val = dpt.one_hot_sequence(val_mesh_ids_array, mesh_vocab_length)
one_hot_mesh_shifted_val = dpt.one_hot_sequence(val_mesh_ids_array_shifted, mesh_vocab_length)

In [15]:
one_hot_reports_train.shape, one_hot_mesh_train.shape, one_hot_mesh_shifted_train.shape

((5148, 6, 11, 1475), (5148, 13, 128), (5148, 13, 128))

## Train Seq-to-Seq Model

In [17]:
from utils.custom_metrics import recall, precision, binary_accuracy
from utils.rnn_textsum_models import HierSeq2Seq

input_dim = len(word_to_id)
output_dim = len(mesh_to_id)
hidden_dim = 512
encoder_emb_dim = 1024
input_word_seq_length = max_num_words
input_sent_seq_length = max_num_sentences
output_seq_length = max_mesh_length
epochs = 20
optimizer = 'adam'
loss='categorical_crossentropy'
batch_size = 128

new_experiment = HierSeq2Seq(epochs=epochs,
                               metrics=['accuracy', binary_accuracy,recall,precision],
                               optimizer=optimizer,
                               loss=loss,
                               batch_size=batch_size, 
                               input_dim=input_dim,
                               output_dim=output_dim,
                               hidden_dim=hidden_dim,
                               encoder_emb_dim=encoder_emb_dim,
                               input_word_seq_length=input_word_seq_length,
                               input_sent_seq_length=input_sent_seq_length,
                               output_seq_length=output_seq_length,
                               verbose=True)
new_experiment.build_model()
new_experiment.model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 6, 11)]      0                                            
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 6, 512)       4658176     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 512), (None, 2099200     time_distributed[0][0]           
____________________________________________________________________________________________

In [ ]:
# create batch generators
# train_batch_generator = dpt.batch_generator_seq2seq(train_token_ids_array, report_vocab_length, train_mesh_ids_array, 
#                                                    train_mesh_ids_array_shifted, mesh_vocab_length, batch_size)

# val_batch_generator = dpt.batch_generator_seq2seq(val_token_ids_array, report_vocab_length, val_mesh_ids_array, 
#                                                    val_mesh_ids_array_shifted, mesh_vocab_length, batch_size)

In [18]:
new_experiment.run_experiment(train_token_ids_array, one_hot_mesh_shifted_train, one_hot_mesh_train, 
                              val_token_ids_array, one_hot_mesh_shifted_val, one_hot_mesh_val)

Train on 5148 samples, validate on 300 samples
Epoch 1/20
5148/5148 [==============================] - 52s 10ms/sample - loss: 2.1934 - accuracy: 0.5957 - binary_accuracy: 0.9950 - recall: 0.4778 - precision: 0.7551 - val_loss: 1.4694 - val_accuracy: 0.6977 - val_binary_accuracy: 0.9966 - val_recall: 0.6213 - val_precision: 0.9241
Epoch 2/20
5148/5148 [==============================] - 44s 9ms/sample - loss: 1.6599 - accuracy: 0.6438 - binary_accuracy: 0.9963 - recall: 0.5682 - precision: 0.9307 - val_loss: 1.3229 - val_accuracy: 0.7108 - val_binary_accuracy: 0.9970 - val_recall: 0.6572 - val_precision: 0.9376
Epoch 3/20
5148/5148 [==============================] - 44s 9ms/sample - loss: 1.5118 - accuracy: 0.6639 - binary_accuracy: 0.9965 - recall: 0.5894 - precision: 0.9465 - val_loss: 1.2167 - val_accuracy: 0.7290 - val_binary_accuracy: 0.9971 - val_recall: 0.6449 - val_precision: 0.9780
Epoch 4/20
5148/5148 [==============================] - 44s 9ms/sample - loss: 1.3564 - accuracy:

In [19]:
model_name = 'hierseq2seq'
model_output_dir = dir + 'trained_models/{}/'.format(model_name)
new_experiment.save_weights_history(model_output_dir)

## Load results of specific experiment

In [20]:
model_name = 'hierseq2seq'
model_output_dir = dir + 'trained_models/{}/'.format(model_name)

In [23]:
import pickle

model_name = 'hierseq2seq'
epochs = 20
encoder_emb_dim = 1024
decoder_emb_dim = 256
hidden_dim = 512

param_fn = 'param_encoderembdim_{}_hiddendim_{}_epochs_{}.pkl'.format(encoder_emb_dim,
                                                                               hidden_dim,
                                                                               epochs)
params = pickle.load(open(model_output_dir + param_fn, 'rb'))

old_experiment = HierSeq2Seq(**params)
old_experiment.build_model()
old_experiment.load_weights_history(model_output_dir)

In [24]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [25]:
def strip_start_end(seq, start_token='start', end_token='end'):
    stripped_seq = []
    for s in seq:
        if s not in [start_token, end_token]:
            stripped_seq.append(s)
    return stripped_seq

In [62]:
def predict_sequence_att(experiment, source, max_seq_len, id_to_mesh, start_token='start', end_token='end'):
    # encode source
    enc_outs, h, c = experiment.encoder_model.predict(source)
    enc_state = [h,c]
    dec_state = enc_state

    # start of sequence input
    in_text = [start_token]

    # integer encoder
    in_seq_ids = dpt.mesh_to_vectors([in_text], dicts_dir=data_dir+'dicts/', 
                   load_dicts=True, save=False)
    # one-hot encode
    in_seq_onehot = dpt.one_hot_sequence(in_seq_ids, mesh_vocab_length)
    in_seq_onehot = np.array(in_seq_onehot)
    in_seq_onehot = in_seq_onehot.reshape(1, 1, in_seq_onehot.shape[-1])
    target_seq = in_seq_onehot
    
    # collect predictions
    output = []
    attention_weights = []
    max_att = []
    for t in range(max_seq_len):
        dec_out, attention, h, c  = experiment.decoder_model.predict([enc_outs] + dec_state + [target_seq])
        dec_state = [h,c]
        # store prediction
        output.append(dec_out[0,0,:])
        dec_ind = np.argmax(dec_out, axis=-1)[0, 0]

        attention_weights.append((dec_ind, attention))
        max_att.append(np.argmax(attention, axis=-1)[0, 0])
        target_seq = dec_out

    predicted_mesh_ids = one_hot_decode(output)
    predicted_mesh = [id_to_mesh[idx] for idx in predicted_mesh_ids]
    
    return predicted_mesh, attention_weights, max_att

In [26]:
def predict_sequence(experiment, report_ids, max_seq_len, id_to_mesh, start_token='start', end_token='end'):
    
    in_text = []
    for t in range(max_seq_len-1):
        # pad input

        in_seq = dpt.pad_sequence(in_text, max_seq_len, 
                                  start_token=start_token, 
                                  end_token=end_token)
        #print(in_seq)
        # integer encoder
        in_seq_ids = dpt.mesh_to_vectors([in_seq], dicts_dir=data_dir+'dicts/', 
                       load_dicts=True, save=False)
        #print(in_seq_ids.shape)
        # one-hot encode
        in_seq_onehot = dpt.one_hot_sequence(in_seq_ids, mesh_vocab_length)
        #print(in_seq_onehot.shape)
        
        prediction = experiment.model.predict([report_ids, in_seq_onehot])
        #print(prediction.shape)
        
        prediction = np.argmax(prediction,axis=2)
        #print(prediction)
        mesh = id_to_mesh[prediction[0][t+1]]
        #print(mesh)
        if mesh == end_token:
            break
        in_text.append(mesh)
    return in_text

In [27]:
for _ in range(10):
    sample = val_df.sample(1)
    true_mesh_caption = list(sample.all_mesh)[0]
    sample_report = list(sample.pad_text_report)[0]
    
    sample_report_ids = []
    for sent in sample_report:
        sent_ids = []
        for token in sent:
            if token in word_to_id.keys():
                sent_ids.append(word_to_id[token])
            else:
                sent_ids.append(word_to_id[unknown_token])
        sample_report_ids.append(sent_ids)

    sample_report_ids = np.array(sample_report_ids)
    sample_report_ids = sample_report_ids.reshape(1, sample_report_ids.shape[0], sample_report_ids.shape[1])
    #print(sample_report_ids.shape)
#     one_hot_sample_report = dpt.one_hot_sequence(sample_report_ids, report_vocab_length)
#     one_hot_sample_report = one_hot_sample_report[np.newaxis,:,:,:]
    #print(one_hot_sample_report.shape)
    prediction = predict_sequence(old_experiment,
                                  sample_report_ids, 
                                  max_mesh_length, 
                                  id_to_mesh,
                                  start_token=start_token,
                                  end_token=end_token)
    predicted_mesh_ids = one_hot_decode(prediction)
    predicted_mesh = [id_to_mesh[idx] for idx in predicted_mesh_ids]
    
    sample_report = strip_start_end(sample_report)
    predicted_mesh = strip_start_end(predicted_mesh)
    
    print('')
    print('Original report: ', sample_report)
    print('True mesh caption: ', true_mesh_caption)
    print('Predicted mesh caption: ', predicted_mesh)


Original report:  [['start', 'heart', 'size', 'cardiomediastinal', 'contours', 'normal', 'end', 'end', 'end', 'end', 'end'], ['start', 'low', 'lung', 'volumes', 'without', 'focal', 'airspace', 'opacity', ',', 'pleural', 'end'], ['start', 'multilevel', 'degenerative', 'changes', 'spine', 'end', 'end', 'end', 'end', 'end', 'end'], ['end', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end'], ['end', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end'], ['end', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end']]
True mesh caption:  ['lung', 'hypoinflation', 'spine', 'degenerative', 'multiple']
Predicted mesh caption:  ['medical_device', 'medical_device', 'medical_device', 'medical_device']

Original report:  [['start', 'heart', 'size', 'normal', 'end', 'end', 'end', 'end', 'end', 'end', 'end'], ['start', 'central', 'left', 'midlung', 'granuloma', 'calcified', 'left', 'hilar', 'adenopathy', 'end', 'end'], ['start', 'bony', 'structures',

## Evaluate BLEU scores on all trian/val/test data

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

def evaluate_model(model, df, report_vocab_length):
    actual, predicted = list(), list()
    bleu1, bleu2, bleu3, bleu4 = list(), list(), list(), list()

    for _, sample in df.iterrows():
        true_mesh_caption = sample.single_mesh
        sample_report = sample.pad_text_report

        sample_report_ids = []
        for sent in report:
            sent_ids = []
            for token in sent:
                if token in word_to_id.keys():
                    sent_ids.append(word_to_id[token])
                else:
                    sent_ids.append(word_to_id[unknown_token])
            sample_report_ids.append(sent_ids)

        sample_report_ids = np.array(sample_report_ids)
        sample_report_ids = sample_report_ids.reshape(1, sample_report_ids.shape[0], sample_report_ids.shape[1])
        #one_hot_sample_report = dpt.one_hot_sequence(sample_report_ids, report_vocab_length)

        #target = predict_sequence(infenc, infdec, one_hot_sample_report, n_steps_out, n_features_out)
        prediction = old_experiment.predict_sequence(sample_report_ids)
        predicted_mesh_ids = one_hot_decode(prediction)
        predicted_mesh = [id_to_mesh[idx] for idx in predicted_mesh_ids]

        # sample_report = strip_start_end(sample_report)
        yhat = strip_start_end(predicted_mesh)
        reference = true_mesh_caption
        
        # calculate BLEU score
        bleu1.append(sentence_bleu([reference], yhat, weights=(1.0, 0, 0, 0)))
        bleu2.append(sentence_bleu([reference], yhat, weights=(0.5, 0.5, 0, 0)))
        bleu3.append(sentence_bleu([reference], yhat, weights=(0.3, 0.3, 0.3, 0)))
        bleu4.append(sentence_bleu([reference], yhat, weights=(0.25, 0.25, 0.25, 0.25)))
    
        # store actual and predicted
        actual.append(reference)
        predicted.append(yhat)
        
    print('BLEU1: ', np.mean(bleu1)*100)
    print('BLEU2: ', np.mean(bleu2)*100)
    print('BLEU3: ', np.mean(bleu3)*100)
    print('BLEU4: ', np.mean(bleu4)*100)
    
    return actual, predicted

In [ ]:
train_actual, train_predicted = evaluate_model(old_experiment, train_df, report_vocab_length)

In [ ]:
val_actual, val_predicted = evaluate_model(old_experiment, val_df, report_vocab_length)

## Evaluate ROUGE scores on all train/val/test data

In [ ]:
import rouge

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                       max_n=4,
                       limit_length=True,
                       length_limit=100,
                       length_limit_type='words',
                       apply_avg='Avg',
                       apply_best='Best',
                       alpha=0.5, # Default F1_score
                       weight_factor=1.2,
                       stemming=True)

In [ ]:
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

In [ ]:
train_hypotheses = [' '.join(p) for p in train_predicted]
train_references = [' '.join(a) for a in train_actual]

scores = evaluator.get_scores(train_hypotheses, train_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

In [ ]:
val_hypotheses = [' '.join(p) for p in val_predicted]
val_references = [' '.join(a) for a in val_actual]

scores = evaluator.get_scores(val_hypotheses, val_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))